In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import AdamW
from datasets import load_dataset
import numpy as np

In [ ]:
data_files = {
    "train": "CNNTrain.csv",
    "test": "CNNTest.csv",
    "validation": "CNNValidation.csv"
}

dataset = load_dataset('csv', data_files=data_files)

In [ ]:
# Preprocessing the data
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
def preprocess_data(examples):
    return tokenizer(examples['article'], truncation=True, padding='max_length', max_length=512)

train_encodings = train_dataset.map(preprocess_data, batched=True)


In [ ]:
# Prepare the dataset for PyTorch
def format_dataset(examples):
    input_ids = torch.tensor(examples['input_ids'])
    attention_mask = torch.tensor(examples['attention_mask'])
    labels = torch.tensor([1 if examples['highlights'] else 0])  # Dummy labels for example
    return TensorDataset(input_ids, attention_mask, labels)


In [ ]:
train_dataset = format_dataset(train_encodings)

# DataLoader
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=8)

# Load model with LoRA and PEFT adaptations
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Adding LoRA layers (simplified example)
class LoRALayer(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = torch.nn.Linear(config.hidden_size, config.hidden_size)
        self.r = torch.nn.Parameter(torch.randn(config.hidden_size, config.hidden_size))

    def forward(self, hidden_states):
        return self.dense(hidden_states) + self.r @ hidden_states

model.roberta.encoder.layer[0].intermediate = LoRALayer(model.config)

In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # num_epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")

In [ ]:
import openai

# Replace 'your_api_key' with your actual API key
api_key = 'sk-proj-7c2PVoOmKXkGcSMMyZ9AT3BlbkFJN17FfVDMnrQQnQ7Bo8fP'
openai.api_key = api_key

In [ ]:
!pip install datasets

import pandas as pd
test_df = pd.DataFrame(dataset['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
test_df.head()

,article,highlights,id
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f


In [ ]:
test_df.drop(columns=['id'], inplace=True)

In [ ]:
test_df.head()

,article,highlights
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...


In [ ]:
test_df['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [ ]:
test_df['highlights'][0]

'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'

In [ ]:
!pip install openai==0.28



In [ ]:
pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [ ]:
import time

# Define a function to generate summaries using the text-davinci-003 model
import openai

def generate_summary_with_openai(article_text):
    prompt = f"Summarize this article:\n{article_text}"

    # Updated method to use the Chat Completion endpoint
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Updated model name for better performance
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=150  # Adjusted token limit based on requirements
    )

    # Adjusted extraction of summary based on the new output format
    summary = response['choices'][0]['message']['content']
    return summary.strip()


# Create an empty column 'model_generated_openai' in test_df to store the generated summaries
test_df['model_generated'] = ""

# Define the batch size and delay
batch_size = 3
delay_seconds = 60  # 1 minute

# Generate summaries in batches
for batch_start in range(0, 25, batch_size):
    batch_end = min(batch_start + batch_size, 25)  # Ensure the end does not exceed 25
    articles_to_process = test_df['article'][batch_start:batch_end]

    # Generate summaries for the batch
    generated_summaries = []
    for article_text in articles_to_process:
        summary = generate_summary_with_openai(article_text)
        generated_summaries.append(summary)

    # Store the generated summaries in the DataFrame
    test_df.loc[batch_start:batch_end-1, 'model_generated'] = generated_summaries

    # Introduce a delay after each batch
    if batch_end < 25:
        print(f"Generated summaries for articles {batch_start+1}-{batch_end}. Waiting for {delay_seconds} seconds before the next batch...")
        time.sleep(delay_seconds)

# Display the updated DataFrame with generated summaries using OpenAI's model
print(test_df[['article', 'model_generated']])


Generated summaries for articles 1-3. Waiting for 60 seconds before the next batch...
Generated summaries for articles 4-6. Waiting for 60 seconds before the next batch...
Generated summaries for articles 7-9. Waiting for 60 seconds before the next batch...
Generated summaries for articles 10-12. Waiting for 60 seconds before the next batch...
Generated summaries for articles 13-15. Waiting for 60 seconds before the next batch...
Generated summaries for articles 16-18. Waiting for 60 seconds before the next batch...
Generated summaries for articles 19-21. Waiting for 60 seconds before the next batch...
Generated summaries for articles 22-24. Waiting for 60 seconds before the next batch...
                                                 article  \
0      (CNN)The Palestinian Authority officially beca...   
1      (CNN)Never mind cats having nine lives. A stra...   
2      (CNN)If you've been following the news lately,...   
3      (CNN)Five Americans who were monitored for thr...   
4 

In [ ]:
test_df.head(25)

,article,highlights,id,model_generated
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01,The Palestinian Authority became the 123rd mem...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,A stray dog in Washington State named Theia mi...
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69,The article provides an overview of Mohammad J...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,Five Americans who were exposed to Ebola in We...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,A Duke student admitted to hanging a noose mad...
5,(CNN)He's a blue chip college basketball recru...,College-bound basketball star asks girl with D...,1b2cc634e2bfc6f2595260e7ed9b42f77ecbb0ce,The article tells the heartwarming story of hi...
6,(CNN)Governments around the world are using th...,Amnesty's annual death penalty report catalogs...,e2706dce6cf26bc61b082438188fdb6e130d9e40,Amnesty International's annual report on the d...
7,"(CNN)Andrew Getty, one of the heirs to billion...",Andrew Getty's death appears to be from natura...,0d3c8c276d079c4c225f034c69aa024cdab7869d,"Andrew Getty, heir to the Getty oil fortune, w..."
8,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...",6222f33c2c79b80be437335eeb3f488509e92cf5,"Tropical storm Maysak, known as Chedeng in the..."
9,"(CNN)For the first time in eight years, a TV l...","Bob Barker returned to host ""The Price Is Righ...",2bd8ada1de6a7b02f59430cc82045eb8d29cf033,"Bob Barker made a special return to ""The Price..."


In [ ]:
test_df['model_generated'][4]

"A Duke student admitted to hanging a noose made of rope from a tree near a student union, leading to a student conduct review and ongoing criminal investigations. Students and faculty protested the incident with a march and forum, denouncing the act as not representative of the university's values. This incident is among recent racist events on college campuses nationwide. Duke University officials condemned the act as cowardly and emphasized that it would not instill fear among the community."

In [ ]:
test_df['model_generated'][5]

"The article tells the heartwarming story of high school basketball star Trey Moses asking freshman Ellie Meredith, who has Down syndrome, to be his prom date. Despite their differences, Trey was drawn to Ellie's kindness and easygoing nature. The prom-posal, which took place during Ellie's P.E. class, has gone viral on social media. Ellie's mother highlighted the importance of inclusive programs like Best Buddies, which have improved Ellie's social experiences. Trey's gesture has been praised by his future coach, family, and teachers, highlighting his caring and compassionate nature. Both Trey and Ellie are excited about prom and are making plans for the special event."

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)
print("BLEU Score for 15 Summaries:", bleu_score)

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
from bert_score import score
# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)

# Print BERT Score
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())

In [ ]:
test_df['model_generated'][5]